In [2]:
import cv2
import numpy as np
import os

# Function to check if file exists
def check_file(file_path, file_type):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Error: {file_type} file '{file_path}' not found!")

# Load YOLO model
try:
    check_file("yolov3.weights", "YOLO Weights")
    check_file("yolov3.cfg", "YOLO Config")

    net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
    print("✅ YOLO files loaded successfully.")
except Exception as e:
    print(f"❌ Error loading YOLO files: {e}")
    exit()

# Get layer names
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Load COCO class labels
try:
    check_file("coco.names", "COCO Names")
    
    with open("coco.names", "r") as f:
        classes = [line.strip() for line in f.readlines()]
    print("✅ COCO names loaded successfully.")
except Exception as e:
    print(f"❌ Error loading COCO names: {e}")
    exit()

# Load image
image_path = "train.png"  # Change this if needed
try:
    check_file(image_path, "Image")
    
    frame = cv2.imread(image_path)
    if frame is None:
        raise ValueError(f"Image '{image_path}' could not be loaded. Check file format.")
    
    print("✅ Image loaded successfully.")
except Exception as e:
    print(f"❌ Error loading image: {e}")
    exit()

height, width, channels = frame.shape

# Prepare the image for the model
blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
net.setInput(blob)
outs = net.forward(output_layers)

class_ids = []
confidences = []
boxes = []

# Extract information from model output
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.7 and classes[class_id] == "person":  # Only detect 'person'
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

# Apply Non-Maximum Suppression (NMS)
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.6, 0.4)

# Count and display detected people
count = 0
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        confidence = confidences[i]
        color = (0, 255, 0)  # Green box
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        cv2.putText(frame, label, (x, y + 30), cv2.FONT_HERSHEY_PLAIN, 1, color, 2)
        count += 1

cv2.putText(frame, f"People Count: {count}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
print(f"👥 People Count: {count}")

# Save the output image
output_image_path = "output_image.png"
try:
    cv2.imwrite(output_image_path, frame)
    print(f"✅ Output image saved at: {output_image_path}")
except Exception as e:
    print(f"❌ Error saving output image: {e}")

✅ YOLO files loaded successfully.
✅ COCO names loaded successfully.
✅ Image loaded successfully.
👥 People Count: 6
✅ Output image saved at: output_image.png
